In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as torch_data
from torchsummary import summary
from torchvision import models
from pycocotools.coco import COCO
import torchvision.transforms as transforms
import json
import skimage.io as io
import matplotlib.pyplot as plt
import numpy as np

import pytorch_lightning as pl

import os, sys
sys.path.append(os.path.abspath("../"))

%load_ext autoreload
%autoreload 2


from models.base_line.model import BaseLine
from models.autoencoders.components import ConvBlock
from models.GANs.model import ConstrastDCGAN

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
class ToyDataset(torch_data.Dataset):
    def __init__(self, len=64):
        super().__init__()
        self.len = len
    
    def __getitem__(self, idx):
        return torch.ones(3, 128, 128), torch.ones(3, 128, 128)*2, torch.zeros(1)

    def __len__(self):
        return self.len

In [8]:
gen_train_dl    = torch_data.DataLoader(torch.ones(8, 3, 128, 128), batch_size=4)
disc_train_dl   = torch_data.DataLoader(torch.ones(50, 3, 64, 64), batch_size=4)
ae_train_dl     = torch_data.DataLoader(ToyDataset(8), batch_size=4)

In [9]:
net = BaseLine(
    512*2*2, 512,
    ae_train_dl=ae_train_dl,
    disc_train_dl=disc_train_dl,
    gen_train_dl=gen_train_dl
)

In [10]:
trainer = pl.Trainer(reload_dataloaders_every_epoch=True, max_epochs=2)
trainer.fit(net)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name               | Type           | Params
------------------------------------------------------
0 | ae                 | ResUnetAE      | 72.8 M
1 | conv_input_decoder | ConvBlock      | 75.5 M
2 | bridge_union       | Sequential     | 47.2 M
3 | gan                | ConstrastDCGAN | 30.7 M
------------------------------------------------------
226 M     Trainable params
0         Non-trainable params
226 M     Total params
904.730   Total estimated model params size (MB)
Epoch 0:   0%|          | 0/2 [00:00<?, ?it/s] 